In [ ]:
!pip install transformers

In [ ]:
import transformers

PIPELINE is a function in transformers, that has a pre processor, model, and post processor


In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis")
classifier(["EU countries push for swift trade with trump",
            "Gold prices are falling"])

**BEHIND THE PIPELINE**



*   preprocessing needs to be done in exactly the same way as when the model was pretrained
*   To do this, we use the AutoTokenizer class and its from_pretrained() method
*   Using the checkpoint name of our model, it will automatically fetch the data associated with the model’s tokenizer and cache i



In [ ]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_inputs = [
    "You are master of your soul ",
    "Captain of your life"
]
inputs = tokenizer(raw_inputs, padding = True, truncation = True, return_tensors = "pt")
print(inputs)

to download a pretrained method Transformers provides an AutoModel class which also has a from_pretrained() method

In [ ]:
from transformers import AutoModel
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

This architecture contains only the base Transformer module:
--given some inputs, it outputs what we’ll call hidden states, also known as features. For each model input, we’ll retrieve a high-dimensional vector representing the contextual understanding of that input by the Transformer model.
A high-dimensional vector?
The vector output by the Transformer module is usually large. It generally has three dimensions:

Batch size: The number of sequences processed at a time (2 in our example).
Sequence length: The length of the numerical representation of the sequence (16 in our example).
Hidden size: The vector dimension of each model input.

While these hidden states can be useful on their own, they’re usually inputs to another part of the model, known as the head. The different tasks could have been performed with the same architecture, but each of these tasks will have a different head associated with it.

In [ ]:
output = model(**inputs)
print(output.last_hidden_state.shape)

The output of the Transformer model is sent directly to the model head to be processed.

In [ ]:
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
print(outputs.logits.shape)
print(outputs.logits)

In [ ]:
import torch

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

In [ ]:
model.config.id2label

Now we can conclude that the model predicted the following:

First sentence: NEGATIVE: 0.0402, POSITIVE: 0.9598
Second sentence: NEGATIVE: 0.9995, POSITIVE: 0.0005